In [1]:
import os
path_pipest = os.path.abspath('./')
n=0
while (not os.path.basename(path_pipest)=='pipest') and (n<6):
    path_pipest=os.path.dirname(path_pipest)
    n+=1 
if not os.path.basename(path_pipest)=='pipest':
    raise ValueError("path_pipest not found. Instead: {}".format(path_pipest))
path_sdhawkes=path_pipest+'/sdhawkes'
path_lobster=path_pipest+'/lobster'
path_lobster_data=path_lobster+'/data'
path_lobster_pyscripts=path_lobster+'/py_scripts'
path_models=path_pipest+'/models'
path_tests=path_pipest+'/tests'
path_saved_tests=path_tests+'/saved_tests'

In [2]:
import time
import sys
sys.path.append(path_sdhawkes+'/resources/')
sys.path.append(path_sdhawkes+'/modelling/')
sys.path.append(path_lobster_pyscripts+'/')

In [3]:
import pickle
import numpy as np
import pandas as pd
import scipy
from scipy.stats import dirichlet as scipy_dirichlet
import timeit

In [4]:
import model as sd_hawkes_model
import lob_model
import computation
import simulation
# import goodness_of_fit
# import mle_estimation as mle_estim
# import uncertainty_quant

openmp.omp_get_max_threads(): 4
openmp.omp_get_max_threads(): 4


In [5]:
symbol = "INTC"
date = "2019-01-23"
time_window = "45000-48600"

In [6]:
# with open(path_models+'/{}/{}_{}/{}_sdhawkes_{}_{}'.format(symbol, symbol, date,symbol, date, time_window),'rb') as source:
#         model=pickle.load(source)
# try:
#     with open(path_models+'/{}/{}_sdhawkes_{}_{}'.format(symbol, symbol, date, time_window),'rb') as source:
#         model=pickle.load(source)
# except FileNotFoundError:
#     with open(path_models+'/{}/{}_{}/{}_sdhawkes_{}_{}'.format(symbol, symbol, date, symbol, date, time_window),'rb') as source:
#         model=pickle.load(source)
# print(model.name_of_model)
# print('Date and time of initilisation: {}'.format(model.datetime_of_initialisation))
# print('number_of_event_types: {}; number_of_states: {}'.format(
#     model.number_of_event_types, model.number_of_states))
# print('Dimension of dataset: {}'.format((model.data.messagefile.shape)))

In [7]:
list_of_n_states=[3,5]
n_events = 4 
n_levels = 2

model = sd_hawkes_model.SDHawkes(number_of_event_types=n_events,
                 number_of_lob_levels=n_levels,
                 volume_imbalance_upto_level=2,
                 list_of_n_states=list_of_n_states
                )

tot_n_states=model.state_enc.tot_n_states

# The base rates $\nu$
nus = 0.002*np.random.randint(low=2,high=6,size=n_events)
# The impact coefficients $\alpha$
alphas = np.random.uniform(0.0002,0.2435,size=(n_events, tot_n_states, n_events)).astype(np.float)
# The decay coefficients $\beta$
betas = np.random.uniform(1.265,1.805,size=(n_events, tot_n_states, n_events)).astype(np.float)
model.set_hawkes_parameters(nus,alphas,betas)
# The transition probabilities $\phi$
phis = model.state_enc.generate_random_transition_prob(n_events=n_events).astype(np.float)
model.set_transition_probabilities(phis)
#     sd_model.enforce_symmetry_in_transition_probabilities()
# The Dirichlet parameters $\kappa$
gammas = np.random.lognormal(size=(tot_n_states,2*n_levels))
model.set_dirichlet_parameters(gammas, use_optimisation=False)


Hawkes parameters have been set
SDHawkes: asymmetry in transition_probabilities = 21.797203221746148
Transition probabilities have been set
Dirichlet parameters have been set


In [8]:
# model.volume_enc.store_param_for_rejection_sampling(epsilon=1.0e-00)

In [9]:
model.volume_enc.rejection_sampling.volimb_limits

array([-1. , -0.6, -0.2,  0.2,  0.6,  1. ])

In [10]:
model.volume_enc.rejection_sampling.is_target_equal_to_proposal

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [11]:
model.volume_enc.prob_volimb_constraint

array([0.22732273, 0.47444744, 0.33173317, 0.0340034 , 0.55275528,
       0.47934793, 0.14021402, 0.41344134, 0.3230323 , 0.41834183,
       0.48174817, 0.08210821, 0.08830883, 0.12221222, 0.74457446])

In [12]:
model.volume_enc.rejection_sampling.prob_constraint_physical

array([0.22732273, 0.47444744, 0.33173317, 0.0340034 , 0.55275528,
       0.47934793, 0.14021402, 0.41344134, 0.3230323 , 0.41834183,
       0.48174817, 0.08210821, 0.08830883, 0.12221222, 0.74457446])

In [13]:
model.volume_enc.rejection_sampling.prob_constraint_proposal

array([0.10511051, 0.25052505, 0.29392939, 0.24892489, 0.10191019,
       0.10591059, 0.25132513, 0.30173017, 0.26272627, 0.10851085,
       0.10441044, 0.24492449, 0.29912991, 0.25072507, 0.10611061])

In [14]:
model.volume_enc.rejection_sampling.inverse_bound

array([1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 3.2535601e+04])

In [15]:
state=14
num_of_st2=model.volume_enc.rejection_sampling.num_of_st2
st2=state%num_of_st2
volimb_limits=model.volume_enc.rejection_sampling.volimb_limits
l, u = volimb_limits[st2], volimb_limits[1+st2]
gamma=model.volume_enc.rejection_sampling.target_dir_param
gamma_tilde=model.volume_enc.rejection_sampling.proposal_dir_param
alpha=model.volume_enc.rejection_sampling.difference_of_dir_params
inverseM=model.volume_enc.rejection_sampling.inverse_bound
is_equal=model.volume_enc.rejection_sampling.is_target_equal_to_proposal
volimb_limits=model.volume_enc.rejection_sampling.volimb_limits
volimb_upto_level=model.volume_enc.rejection_sampling.volimb_upto_level
upto_lim=1+2*volimb_upto_level

In [16]:
physical_prob=computation.compute_probability_of_volimb_constraint(
    volimb_upto_level, 1, 0, volimb_limits[st2:st2+2], gamma[state,:], N_samples=99999
)
proposal_prob=computation.compute_probability_of_volimb_constraint(
    volimb_upto_level, 1, 0, volimb_limits[st2:st2+2], gamma_tilde[state,:], N_samples=99999
)
print("gamma={}".format(gamma[state,:]))
print("gamma_tilde={}".format(gamma_tilde[state,:]))
print("bounds={}".format(volimb_limits[st2:st2+2]))
print("l={}, u={}".format(l,u))
print("physical_prob={}".format(physical_prob))
print("proposal_prob={}".format(proposal_prob))

gamma=[ 1.45809052 10.37258471  1.21516695  4.32430453]
gamma_tilde=[1. 1. 1. 1.]
bounds=[0.6 1. ]
l=0.6000000000000001, u=1.0
physical_prob=0.7384573845738457
proposal_prob=0.1047710477104771


In [17]:
sample=scipy_dirichlet.rvs(gamma[state,:])[0,:]
sample_tilde=scipy_dirichlet.rvs(gamma_tilde[state,:])[0,:]
qi=np.sum(sample[1:upto_lim:2]-sample[0:upto_lim:2])
qi_tilde=np.sum(sample_tilde[1:upto_lim:2]-sample_tilde[0:upto_lim:2])
accept=((qi>=l)&(qi<u))
accept_tilde=((qi_tilde>=l)&(qi_tilde<u))
print(sample)
print(sample_tilde)
print("accept: {} ".format(accept))
print("accept_tilde: {}".format(accept_tilde))

[0.13441743 0.57268467 0.1058971  0.18700081]
[0.0312007  0.0128749  0.68871891 0.26720548]
accept: False 
accept_tilde: False


In [18]:
upto_lim

5

In [19]:
num_rejconstr=[]

In [20]:
%%timeit -n 2 -r 1
num_rejconstr.append(simulation.naive_sample(state,
             gamma,
             num_of_st2, volimb_limits, upto_lim))

133 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 2 loops each)


In [21]:
print(np.mean(np.array(num_rejconstr)))
print(num_rejconstr)

0.5
[0, 1]


In [22]:
num_rejconstr=[]

In [23]:
%%timeit -n 2 -r 1
num_rejconstr.append(simulation.sample_volumes(state,
               gamma_tilde,
               alpha, inverseM,
               is_equal, num_of_st2, volimb_limits, upto_lim))

5.47 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 2 loops each)


In [24]:
print(np.mean(np.array(num_rejconstr)))
print(num_rejconstr)

76.0
[95, 57]


In [25]:
model.volume_enc.rejection_sampling.inverse_bound

array([1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 3.2535601e+04])

In [26]:
# model.state_enc.df_state_enc

In [27]:
# model.goodness_of_fit.qq_plot_residuals(index_of_first_event_type=1)
# print(model.goodness_of_fit.ad_test_on_residuals())

In [28]:
# model.mle_estim.goodness_of_fit.qq_plot_residuals(index_of_first_event_type=1)
# print(model.mle_estim.goodness_of_fit.ad_test_on_residuals())

In [29]:
def sample_volumes(state, proposal_dir_param,
     difference_of_dir_params, inverse_bound,
     is_target_equal_to_proposal,
     num_of_st2, volimb_limits, upto_lim = 5, maxiter= 999999
):
    st_2 = state%num_of_st2
    lower_bound = volimb_limits[st_2]
    upper_bound = volimb_limits[1+st_2]
    gamma_tilde = proposal_dir_param[state,:]
    delta_gamma = difference_of_dir_params[state,:]
    K = inverse_bound[state]
    val, u, volimb = 0.0, 0.0, 0.0
    reject = 1
    count = 0
    num_rej=0
    num_rejconstr = 0
    num_rejbnd = 0
    if is_target_equal_to_proposal[state]:
        while (reject) & (count<=maxiter):
            count += 1
            sample = scipy_dirichlet.rvs(gamma_tilde)[0,:]
            vol_imb = np.sum(sample[1:upto_lim:2] - sample[0:upto_lim:2])
            if (lower_bound <= vol_imb)&(vol_imb<=upper_bound):
                reject = 0
                passed[count-1]=1
            else:
                num_rejconstr+=1
    else:
        while (reject) & (count<=maxiter):
            count += 1
            sample = scipy_dirichlet.rvs(gamma_tilde)[0,:]
            vol_imb = np.sum(sample[1:upto_lim:2] - sample[0:upto_lim:2])
            if ((lower_bound <= vol_imb)&(vol_imb<=upper_bound)):
                u = np.random.uniform(low=0.0, high=1.0)
                val = np.prod(np.power(sample,delta_gamma))
                if u < K*val:
                    reject = 0
                else:
                    num_rejbnd+=1
                    num_rej+=1
            else:
                num_rejconstr+=1
                num_rej+=1
    return num_rejconstr, num_rejbnd

In [30]:
def naive_sample(state, dir_param,
     num_of_st2, volimb_limits, upto_lim = 5, maxiter= 999999
 ):
    st_2 = state%num_of_st2
    lower_bound = volimb_limits[st_2]
    upper_bound = volimb_limits[1+st_2]
    gamma = dir_param[state,:]
    vol_imb = 0.0
    reject = 1
    count = 0
    num_rejconstr = 0
    while (reject) & (count<=maxiter):
        count += 1
        sample = scipy_dirichlet.rvs(gamma)[0,:]
        vol_imb = np.sum(sample[1:upto_lim:2] - sample[0:upto_lim:2])
        if ((lower_bound <= vol_imb)&(vol_imb<=upper_bound)):
            reject = 0
        else:
            num_rejconstr+=1
    return num_rejconstr


In [31]:
def compute_prob(state, dir_param,
     num_of_st2, volimb_limits, upto_lim = 5, N_samples= 999999
 ):
    st_2 = state%num_of_st2
    lower_bound = volimb_limits[st_2]
    upper_bound = volimb_limits[1+st_2]
    gamma = dir_param[state,:]
    vol_imb = 0.0
    passed = np.zeros(N_samples,dtype=np.int)
    count = 0
    for n in range(N_samples):
        sample = scipy_dirichlet.rvs(gamma)[0,:]
        vol_imb = np.sum(sample[1:upto_lim:2] - sample[0:upto_lim:2])
        if ((lower_bound <= vol_imb)&(vol_imb<=upper_bound)):
            #count+=1 
            passed[n]=1
    prob=float(np.sum(passed))/float(N_samples)
    return passed,prob#float(count)/float(N_samples)

In [32]:
N_samples=9999
physpassed,physprob=compute_prob(state,gamma,num_of_st2,volimb_limits,upto_lim,N_samples)
proppassed,propprob=compute_prob(state,gamma_tilde,num_of_st2,volimb_limits,upto_lim,N_samples)
print("physprob={}".format(physprob))
print("propprob={}".format(propprob))

physprob=0.7432743274327432
propprob=0.09950995099509952


In [33]:
def expectation_constr(gamma,l,u,uplim=5):
    v=np.sum(gamma[1:uplim:2]-gamma[0:uplim:2])/np.sum(gamma)
    return (max(0.0,v-u)+max(0.0,l-v))**2

In [34]:
# def compute_proposal_param(gamma,l=-0.2,u=0.2,c=1.0,uplim=5,maxiter=1000):
#     def objfun(rho):
#         return computation.compute_maximum_unnormalised_pseudo_dirichlet_density(gamma-rho)\
#         +c*expectation_constr(rho)
#     def expectation_constr(rho):
#         v=np.sum(rho[1:uplim:2]-rho[0:uplim:2])/np.sum(rho)
#         return (max(0.0,v-u)+max(0.0,l-v))**2
#     bounds=[(1.0e-8,gamma[k]) for k in range(len(gamma))]
#     res=scipy.optimize.minimize(
#         objfun,0.95*gamma,
#         method='TNC',jac=False,
#         bounds=bounds,options={'maxiter': maxiter})
#     rho=res['x']
#     return rho

In [84]:
from scipy.special import loggamma as LogGamma

In [90]:
def objfun(rho,gamma):
    alpha=gamma-rho
    def logM():
        return np.sum(alpha*np.log(alpha/np.sum(alpha)))
    return logB(rho)+logM()
def logB(rho):
    return np.sum(LogGamma(rho))-LogGamma(np.sum(rho))
def find_rho(gamma,maxiter=1000, tol=1.0e-6):
    bounds=tuple([(1.0e-8,(1.0-tol)*gamma[k]) for k in range(len(gamma))])
    res=scipy.optimize.minimize(
        objfun,0.995*gamma,args=(gamma),
        method='TNC',jac=False,
        bounds=bounds,options={'maxiter': maxiter})
    return res

In [97]:
gamma=np.random.uniform(low=0.0, high=10.0, size=(4,))
print('gamma: {}'.format(gamma))
print('target: {}'.format(logB(gamma)))
res=find_rho(gamma)
print('minimisation results:\n {}'.format(res))
better=res['fun']<logB(gamma)
print("better: {}".format(better))

gamma: [9.88880007 2.64481406 3.87733993 9.4349304  8.93254443 7.12686851
 2.50572472 8.93807836 4.62915631 7.01033018 9.06058593]
target: -169.3423178856035
minimisation results:
      fun: -169.34142303729655
     jac: array([ 0.04894787,  0.01891465, -0.26934117, -0.10657288, -0.08635936,
       -0.06461676, -0.27312694, -0.16920865, -0.08489849,  0.03674359,
        0.02226841])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 186
     nit: 31
  status: 1
 success: True
       x: array([9.88732133, 2.64446306, 3.8766064 , 9.43328632, 8.93102374,
       7.12569873, 2.50528465, 8.93642525, 4.62841139, 7.00929169,
       9.05920106])
better: False
